<a href="https://colab.research.google.com/github/szdziedzic/intent-classification-for-polish-language/blob/main/colab_experiments/herbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.9 MB/s eta 0:00:00


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.8 MB/s eta 0:00:00


In [3]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=e96ffe01819124f0d515f8015c4b0ce923b3bb08e4be376441ad1150edb6058f
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [55]:
from datasets import load_dataset

train_dataset = load_dataset("AmazonScience/massive", "pl-PL", split='train')
print(train_dataset[0])

{'id': '1', 'locale': 'pl-PL', 'partition': 'train', 'scenario': 16, 'intent': 48, 'utt': 'obudź mnie o dziewiątej rano w piątek', 'annot_utt': 'obudź mnie o [time : dziewiątej rano] w [date : piątek]', 'worker_id': '9', 'slot_method': {'slot': ['time', 'date'], 'method': ['translation', 'translation']}, 'judgments': {'worker_id': ['21', '11', '5'], 'intent_score': [1, 1, 1], 'slots_score': [1, 1, 1], 'grammar_score': [4, 4, 4], 'spelling_score': [2, 2, 2], 'language_identification': ['target', 'target', 'target']}}


In [5]:
val_dataset = load_dataset("AmazonScience/massive", "pl-PL", split='validation')
print(val_dataset[0])

{'id': '11', 'locale': 'pl-PL', 'partition': 'dev', 'scenario': 8, 'intent': 40, 'utt': 'wyłącz światła proszę', 'annot_utt': 'wyłącz światła proszę', 'worker_id': '6', 'slot_method': {'slot': [], 'method': []}, 'judgments': {'worker_id': ['2', '5'], 'intent_score': [1, 1], 'slots_score': [1, 1], 'grammar_score': [4, 4], 'spelling_score': [2, 2], 'language_identification': ['target', 'target']}}


In [6]:
test_dataset = load_dataset("AmazonScience/massive", "pl-PL", split='test')
print(test_dataset[0])

{'id': '0', 'locale': 'pl-PL', 'partition': 'test', 'scenario': 16, 'intent': 48, 'utt': 'obudź mnie o piątej rano w tym tygodniu', 'annot_utt': 'obudź mnie o [time : piątej rano] [date : w tym tygodniu]', 'worker_id': '14', 'slot_method': {'slot': ['time', 'date'], 'method': ['translation', 'translation']}, 'judgments': {'worker_id': ['5', '21', '11'], 'intent_score': [1, 1, 1], 'slots_score': [1, 1, 1], 'grammar_score': [4, 4, 4], 'spelling_score': [2, 2, 2], 'language_identification': ['target', 'target', 'target']}}


In [7]:
from transformers import AutoTokenizer, AutoModel

model_names = {
    "herbert-klej-cased-v1": {
        "tokenizer": "allegro/herbert-klej-cased-tokenizer-v1",
        "model": "allegro/herbert-klej-cased-v1",
    },
    "herbert-base-cased": {
        "tokenizer": "allegro/herbert-base-cased",
        "model": "allegro/herbert-base-cased",
    },
    "herbert-large-cased": {
        "tokenizer": "allegro/herbert-large-cased",
        "model": "allegro/herbert-large-cased",
    },
}

tokenizer = AutoTokenizer.from_pretrained(model_names["herbert-base-cased"]["tokenizer"])
model = AutoModel.from_pretrained(model_names["herbert-base-cased"]["model"])

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.sso.sso_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.sso.sso_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
output = model(
    **tokenizer.batch_encode_plus(
        [
            (
                "A potem szedł środkiem drogi w kurzawie, bo zamiatał nogami, ślepy dziad prowadzony przez tłustego kundla na sznurku.",
                "A potem leciał od lasu chłopak z butelką, ale ten ujrzawszy księdza przy drodze okrążył go z dala i biegł na przełaj pól do karczmy."
            )
        ],
        padding="longest",
        add_special_tokens=True,
        return_tensors="pt",
    )
)

In [9]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0156, -0.0107,  0.0548,  ..., -0.2931,  0.0463, -0.4071],
         [-0.1879, -0.2544, -0.1516,  ..., -0.2208,  0.2126,  0.2200],
         [-0.1578, -0.0876, -0.4259,  ...,  0.0314,  0.6145, -0.2184],
         ...,
         [ 0.0454, -0.2671, -0.1461,  ...,  0.3076,  0.8337,  0.3293],
         [ 0.0229,  0.1212,  0.0126,  ..., -0.4731,  0.1688,  0.3738],
         [-0.1684, -0.0869, -0.4155,  ...,  0.0268,  0.6186, -0.2000]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 2.3978e-01,  7.7380e-02, -1.0388e-01,  9.9938e-01, -4.3555e-02,
          1.6402e-01,  9.9964e-01, -9.9971e-01, -9.9978e-01,  6.9485e-01,
          9.9686e-01,  8.6258e-02,  9.9989e-01, -7.4082e-01,  4.8495e-02,
          6.2282e-02, -9.9989e-01,  2.9818e-03,  3.0078e-02,  9.9358e-01,
          9.9906e-01, -9.3512e-01, -2.7758e-02, -1.6144e-01,  1.2141e-01,
         -9.3916e-01, -8.6540e-01, -9.4444e-01, -3.4983e-02, -9.946

In [10]:
import torch
import torch.nn as nn

In [11]:
MASSIVE_INTENTS = ['datetime_query', 'iot_hue_lightchange', 'transport_ticket', 'takeaway_query', 'qa_stock',
            'general_greet', 'recommendation_events', 'music_dislikeness', 'iot_wemo_off', 'cooking_recipe',
            'qa_currency', 'transport_traffic', 'general_quirky', 'weather_query', 'audio_volume_up',
            'email_addcontact', 'takeaway_order', 'email_querycontact', 'iot_hue_lightup',
            'recommendation_locations', 'play_audiobook', 'lists_createoradd', 'news_query',
            'alarm_query', 'iot_wemo_on', 'general_joke', 'qa_definition', 'social_query',
            'music_settings', 'audio_volume_other', 'calendar_remove', 'iot_hue_lightdim',
            'calendar_query', 'email_sendemail', 'iot_cleaning', 'audio_volume_down',
            'play_radio', 'cooking_query', 'datetime_convert', 'qa_maths', 'iot_hue_lightoff',
            'iot_hue_lighton', 'transport_query', 'music_likeness', 'email_query', 'play_music',
            'audio_volume_mute', 'social_post', 'alarm_set', 'qa_factoid', 'calendar_set',
            'play_game', 'alarm_remove', 'lists_remove', 'transport_taxi', 'recommendation_movies',
            'iot_coffee', 'music_query', 'play_podcasts', 'lists_query']

In [12]:
linear_layer = nn.Linear(model.config.hidden_size, len(MASSIVE_INTENTS))

In [13]:
logits = linear_layer(output.pooler_output)

In [14]:
logits

tensor([[ 0.2139,  0.5687, -0.5456,  0.0432, -0.4006,  0.0620,  0.1333, -0.0575,
          0.5194, -0.0792,  0.6794,  0.1729,  0.1662, -0.4371, -0.1340, -0.2299,
          0.5505, -0.4824,  0.0124,  0.8338,  0.2838,  0.4190,  0.3075,  0.1293,
         -0.3565,  0.1452, -0.5870, -0.2914, -0.8165, -0.0420,  0.5777,  0.2592,
          0.2858, -0.1924,  0.6005, -0.1472,  0.9581,  0.1958,  0.6944, -0.1494,
         -0.0097,  0.1752, -0.2053,  0.5870, -0.8055,  0.3106, -0.4703,  0.7061,
         -0.1790,  0.1375,  0.3922, -0.4006, -0.3386, -0.1184, -0.6119,  0.2734,
          0.1850, -0.3758,  0.0851,  0.0999]], grad_fn=<AddmmBackward0>)

In [15]:
probabilities = torch.softmax(logits, dim=-1)

In [16]:
probabilities

tensor([[0.0180, 0.0257, 0.0084, 0.0152, 0.0097, 0.0155, 0.0166, 0.0137, 0.0244,
         0.0134, 0.0287, 0.0173, 0.0172, 0.0094, 0.0127, 0.0115, 0.0252, 0.0090,
         0.0147, 0.0334, 0.0193, 0.0221, 0.0198, 0.0165, 0.0102, 0.0168, 0.0081,
         0.0109, 0.0064, 0.0139, 0.0259, 0.0188, 0.0193, 0.0120, 0.0265, 0.0125,
         0.0379, 0.0177, 0.0291, 0.0125, 0.0144, 0.0173, 0.0118, 0.0261, 0.0065,
         0.0198, 0.0091, 0.0294, 0.0121, 0.0167, 0.0215, 0.0097, 0.0104, 0.0129,
         0.0079, 0.0191, 0.0175, 0.0100, 0.0158, 0.0161]],
       grad_fn=<SoftmaxBackward0>)

In [17]:
predicted_intent_index = torch.argmax(probabilities, dim=-1)

In [18]:
predicted_intent_index

tensor([36])

In [19]:
predicted_intent = MASSIVE_INTENTS[predicted_intent_index]

In [20]:
predicted_intent

'play_radio'

In [21]:
output.pooler_output.shape

torch.Size([1, 768])

In [56]:
from torch.utils.data import Dataset, DataLoader, Subset

class MassiveDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        example = self.dataset[index]
        return example["utt"], example["intent"]

train_dataloader = DataLoader(Subset(MassiveDataset(train_dataset), range(100)), batch_size=32, shuffle=True)
val_dataloader = DataLoader(Subset(MassiveDataset(val_dataset), range(50)), batch_size=32, shuffle=True)
test_dataloader = DataLoader(Subset(MassiveDataset(test_dataset), range(50)), batch_size=32, shuffle=True)

In [49]:
import torch.nn.functional as F

class MASSIVEIntentClassifier(nn.Module):
  def __init__(self):
        super().__init__()
        self.linear = nn.Linear(model.config.hidden_size, len(MASSIVE_INTENTS))

  def forward(self, x):
      x = self.linear(x)
      return F.softmax(x)

In [24]:
MASSIVEIntentClassifier()(output.pooler_output)

<ipython-input-23-46b871e21d0e>:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(x)


tensor([37])

In [25]:
from sklearn.metrics import accuracy_score

In [57]:
next(iter(train_dataloader))

[('jakie są najnowsze wiadomości tvn',
  'co nowego w wiadomościach tvn',
  'piosenka którą grasz jest niesamowita',
  'co dzisiaj dzieje się w piłce nożnej',
  'co to za album z którego pochodzi ta muzyka',
  'odtwórz daft punk',
  'czas spać olly',
  'proszę zagraj moją ulubioną muzykę',
  'wystartują moją playlistę jazz',
  'odtwórz moją playlistę gatunku jazz',
  'pokaż mi godzinę w moskwie',
  'pauza przez dziesięć sekund',
  'nastaw alarm na za dwie godziny od teraz',
  'znajdź tajskie jedzenie na wynos w pobliżu rynku',
  'przyciemnij światła w kuchni olly',
  'wymień mi najwyżej oceniane opcje chińskiego jedzenia z dostawą',
  'odkurz dom olly',
  'przetasuj tę playlistę',
  'włącz robota sprzątającego',
  'olly zatrzymaj na dziesięć sekund',
  'olly rozśmiesz mnie',
  'wyłącz światło w łazience',
  'hej włącz robot sprzątający',
  'czy mogę zamówić obiad na wynos z bombardino',
  'olly przyciemnij światła w korytarzu',
  'jaka jest pogoda na ten tydzień',
  'odkurz dom',
  'na

In [59]:
from tqdm import tqdm

number_of_epochs = 50

intent_clf = MASSIVEIntentClassifier()
intent_clf = intent_clf.cuda()
loss_fn = nn.CrossEntropyLoss()

opt = torch.optim.Adam(
  intent_clf.parameters(),
  lr=3e-4,
)
epoch_progress = tqdm(list(range(number_of_epochs)))

history = []
for i in epoch_progress:
    train_loss = 0
    y_train_predicted = []
    y_train_true = []
    intent_clf.train()
    for iteration, (X_train, y_train) in enumerate(train_dataloader):
        y_train = y_train.cuda()
        opt.zero_grad()
        output = model(
            **tokenizer.batch_encode_plus(
                X_train,
                padding="longest",
                add_special_tokens=True,
                return_tensors="pt",
            )
        )
        pooler_output = output.pooler_output
        pooler_output = pooler_output.cuda()
        y_pred = intent_clf(pooler_output)
        loss = loss_fn(y_pred, y_train)
        loss.backward()
        opt.step()
        train_loss += loss.item()
        y_train_predicted.extend(torch.argmax(y_pred, dim=-1).cpu().numpy())
        y_train_true.extend(y_train.cpu().numpy())


    val_loss = 0
    y_predicted = []
    y_true = []

    intent_clf.eval()
    with torch.no_grad():
        for iteration, (X_val, y_val) in enumerate(val_dataloader):
            y_val = y_val.cuda()
            output = model(
                **tokenizer.batch_encode_plus(
                    X_val,
                    padding="longest",
                    add_special_tokens=True,
                    return_tensors="pt",
                )
            )
            pooler_output = output.pooler_output
            pooler_output = pooler_output.cuda()
            y_pred = intent_clf(pooler_output)
            loss = loss_fn(y_pred, y_val)
            val_loss += loss.item()
            y_true.extend(y_val.cpu().numpy())
            y_predicted.extend(torch.argmax(y_pred, dim=-1).cpu().numpy())
    train_acc = accuracy_score(y_train_true, y_train_predicted)
    val_acc = accuracy_score(y_true, y_predicted)
    epoch_progress.set_description(f'#Epoch: {i}, train loss: {train_loss:.2f}, train_acc: {train_acc:.2f}, val loss: {val_loss:.2f}, val_acc: {val_acc:.2f}')
    history.append({"e": i, "train_acc": train_acc, "val_acc": val_acc})

  0%|          | 0/50 [00:00<?, ?it/s]<ipython-input-49-eb3975bb5354>:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
#Epoch: 0, train loss: 16.37, train_acc: 0.04, val loss: 8.18, val_acc: 0.04:   2%|▏         | 1/50 [00:18<15:07, 18.52s/it]<ipython-input-49-eb3975bb5354>:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
#Epoch: 1, train loss: 16.34, train_acc: 0.05, val loss: 8.17, val_acc: 0.04:   4%|▍         | 2/50 [00:31<12:22, 15.47s/it]<ipython-input-49-eb3975bb5354>:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
#Epoch: 2, train loss: 16.30, train_acc: 0.05, val loss: 8.16, val_acc: 0.04:   6%|▌         | 3/50 [00:46<11:42, 14.95s/it]<ipython-input-49-eb3975bb5354>:10: UserWarning: Impl

KeyboardInterrupt: ignored